# Introduction to SQL

## Overview of the SQL query language

The **Structured Query Language (SQL)** has several parts:

1. **Data-definition language (DDL)**; the SQL DDL provides commands for defining relation schemas (e.g. ``student (age, name, sex, ...)``), deleting relations, and modifying relation schemas. Specifically with commands for:
    * **Integrity**; specifying integrity constraints that the data stored in the database must satisfy. Updates that violate integrity constraints are disallowed.
    * **View definition**;
2. **Data-manipulation language (DML)**; the SQL DML provides the ability to query information from the database and to insert tuples into, delete tuples from, and modify tuples in a database. Here, it seems crucial to point out that SQL allows for:
    * **transaction control**; a transaction is a unit of work performed against a database, such as update or delete; SQL contains commands to control whether the transactions are actually concluded or not which is useful to guarantee the integrity of the database, namely via commands such as ``COMMIT`` or ``ROLL BACK``

## Basic data types

Main built-in data types supported:

* ``char(n)``; a fixed-length character string with user-specified length n, if the string is shorter than *n* SQL recycles it with spaces, e.g. if ``char(3)``,"hi" becomes "hi "
* ``varchar(n)``; A variable-length character string with user-specified maximum length;
* ``boolean``; logical values TRUE, FALSE AND UNKNOWN
* ``int``; An integer;
* ``smallint``; 
* ``numeric(p,d)``; a fixed-point number with user-specified precision, defined by the *p* parameter as number of digits of the number, and *d* as the number of the *p* digits to the right of the decimal point.
* ``real, double precision``; Floating-point and double-precision floating-point numbers with machine dependent precision;
* ``float(n)``; A floating-point number wiht precision of at least n digits
* ``date``; defaults to the following format YYYY-MM-DD. For other formats, including time and datetime, [see](https://docs.microsoft.com/en-us/sql/t-sql/data-types/date-transact-sql?view=sql-server-ver15)

## Basic Schema Definition (creating databases and tables)

Before actually writting the code, lets create a temp database. If it already exists delete it.



In [35]:
-- conditional creation of a temporary database
USE master;
GO
IF EXISTS (SELECT * FROM sys.databases WHERE name = 'uni')
    BEGIN
        -- delete the db
        DROP DATABASE uni
    END
ELSE 
    BEGIN
        -- create it
        CREATE DATABASE uni
    END;
GO

-- double check
SELECT name, database_id AS id FROM sys.databases WHERE name = 'uni';
GO

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.715

name,id
uni,7


We define a SQL relation via the <code>CREATE TABLE</code> command. The command below defines the relation department. Notice that integrity constraints are added at the bottom within a list. In this cade we specificy that dept_name is the primary key of the department relation.

In [36]:
-- select the relevant db
USE uni;
GO

-- creat the dept table
CREATE TABLE department (
    dept_name VARCHAR(20),
    building VARCHAR(15),
    budget numeric (12,2),
    PRIMARY KEY (dept_name)
)

-- double-check
SELECT * FROM department;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.007

dept_name,building,budget


Sql <strong>supports a number of different integrity constraints</strong>, namely:

- `FOREIGN KEY (A_1, ..., A_n) REFERENCE s`; recall that foreign key restriction state that the value of attributes `(A_1, ..., A_n)` for any tuple in the relation must correspond to values of the primary key attributes of some tuple in relation s.<br>

<br>

In [37]:
--create another table
CREATE TABLE professors (
    prof_id int,
    prof_name varchar(250),
    salary NUMERIC(8,2),
    dept_name varchar(20),
    PRIMARY KEY (prof_id),
    FOREIGN KEY (dept_name) REFERENCES department
);
GO

SELECT * FROM professors;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.011

prof_id,prof_name,salary,dept_name


- `PRIMARY KEY ((A_1, ..., A_n)`; the primary key attributes are required to be **non-null** and **unique**. The primary key specifications are optional but strongly recommended.

In [38]:
-- another example
CREATE TABLE courses (
    course_id int,
    title varchar(15),
    dept_name varchar(20),
    PRIMARY KEY (course_id),
    FOREIGN KEY (dept_name) REFERENCES department
);
GO

-- primary key can be composed by many fields
CREATE TABLE teaches(
    prof_id int,
    course_id int,
    semester varchar(6),
    year numeric(4,0),
    PRIMARY KEY (prof_id, course_id, semester, year),
    FOREIGN KEY (course_id) REFERENCES courses,
    FOREIGN KEY (prof_id) REFERENCES professors,
);
GO

SELECT * FROM courses;
GO

SELECT * FROM teaches;
GO

Commands completed successfully.

Commands completed successfully.

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.015

course_id,title,dept_name


prof_id,course_id,semester,year


## Droping dabases and tables

It is pretty straightforward.

In [39]:
-- Switch database being used
USE master;
GO
-- droping a database as a whole
DROP DATABASE uni;
GO
-- rebuilding it but without foreign key constraints
CREATE DATABASE uni;
GO
USE uni;
GO
CREATE TABLE courses (
    course_id int,
    title varchar(15),
    dept_name varchar(20),
    PRIMARY KEY (course_id)
);
GO
CREATE TABLE teaches(
    prof_id int,
    course_id int,
    semester varchar(6),
    year numeric(4,0),
    PRIMARY KEY (prof_id, course_id, semester, year)
);
GO
CREATE TABLE professors (
    prof_id int,
    prof_name varchar(250),
    salary NUMERIC(8,2),
    dept_name varchar(20),
    PRIMARY KEY (prof_id),
);
GO

SELECT name, object_id AS id FROM sys.tables;
GO 
-- Drop a table
DROP TABLE courses;
GO
-- check
SELECT name, object_id AS id FROM sys.tables;
GO 
-- Drop two tables
DROP TABLE teaches, professors;
GO
-- check
SELECT name, object_id AS id FROM sys.tables;
GO
-- Switch database being used
USE tempdb;
GO
-- Drop the entire database
DROP DATABASE uni;
GO
-- check
SELECT name FROM sys.databases;
GO  

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Total execution time: 00:00:00.827

name,id
courses,581577110
teaches,613577224
professors,645577338


name,id
teaches,613577224
professors,645577338


name,id


name
master
tempdb
model
msdb
udemy
dbm_project
